In [32]:
# !pip install meshio polyscope

In [33]:
import os
from collections import Counter
import meshio
import numpy as np
import polyscope as ps
import logging

In [34]:
def load_mesh(file_path):
    """Load a mesh file using meshio."""
    mesh = meshio.read(file_path)
    return mesh.points, mesh.cells

In [35]:
def extract_faces(cells):
    """Extract all triangular faces from tetrahedral or other volumetric cells."""
    all_faces = []
    for cell_block in cells:
        if cell_block.type == "tetra":
            for cell in cell_block.data:
                all_faces.extend([
                    tuple(sorted([cell[0], cell[1], cell[2]])),
                    tuple(sorted([cell[0], cell[1], cell[3]])),
                    tuple(sorted([cell[0], cell[2], cell[3]])),
                    tuple(sorted([cell[1], cell[2], cell[3]])),
                ])
        elif cell_block.type == "triangle":
            for cell in cell_block.data:
                all_faces.append(tuple(sorted(cell)))
    return all_faces

In [36]:
def extract_boundary_faces(faces):
    """
    Given a list of triangular faces (as tuples), determine which are on the boundary.
    Boundary faces appear exactly once, while internal faces appear twice.
    Returns a list of boundary face tuples.
    """
    face_count = Counter(faces)
    boundary_faces = [f for f, count in face_count.items() if count == 1]
    return boundary_faces

In [37]:
def calculate_face_area(vertices, face):
    """Calculate the area of a triangular face."""
    v1, v2, v3 = vertices[list(face)]
    # Calculate two edges of the triangle
    edge1 = v2 - v1
    edge2 = v3 - v1
    # Calculate the cross product and its magnitude
    cross_product = np.cross(edge1, edge2)
    area = 0.5 * np.linalg.norm(cross_product)
    return area

In [38]:
def calculate_contact_surface_area(vertices, touching_faces):
    """Calculate the total surface area of touching faces."""
    total_area = 0.0
    for face in touching_faces:
        area = calculate_face_area(vertices, face)
        total_area += area
    return total_area

In [39]:
def main():
    # Update these paths to your mesh files
    file_path1 = os.path.join(os.getcwd(), "../../meshes/rectangle.mesh")
    file_path2 = os.path.join(os.getcwd(), "../../meshes/rectangle.mesh")
    
    if not os.path.exists(file_path1):
        raise FileNotFoundError(f"Mesh 1 file not found: {file_path1}")
    if not os.path.exists(file_path2):
        raise FileNotFoundError(f"Mesh 2 file not found: {file_path2}")
    
    # Load meshes
    vertices1, cells1 = load_mesh(file_path1)
    vertices2, cells2 = load_mesh(file_path2)

    # Define the translation vector
    translation_vector = np.array([12.0, 0.0, 0.0])

    # Translate the second mesh vertices
    vertices2_translated = vertices2 + translation_vector

    # Extract faces for both meshes
    all_faces_1 = extract_faces(cells1)
    all_faces_2 = extract_faces(cells2)

    # Get boundary and internal faces for Mesh 1
    face_count_1 = Counter(all_faces_1)
    boundary_faces_1 = [f for f, count in face_count_1.items() if count == 1]
    internal_faces_1 = [f for f, count in face_count_1.items() if count > 1]

    # Get boundary and internal faces for Mesh 2
    face_count_2 = Counter(all_faces_2)
    boundary_faces_2 = [f for f, count in face_count_2.items() if count == 1]
    internal_faces_2 = [f for f, count in face_count_2.items() if count > 1]

    # Identify touching faces based on the translated mesh
    set_bf1 = set(boundary_faces_1)
    set_bf2 = set(boundary_faces_2)
    touching_faces = list(set_bf1.intersection(set_bf2))

    # Calculate non-touching exterior faces
    external_faces_1 = list(set_bf1.difference(touching_faces))
    external_faces_2 = list(set_bf2.difference(touching_faces))

    # Debug information
    logging.info(f"Mesh 1: {len(vertices1)} vertices, {len(all_faces_1)} total faces")
    logging.info(f"Mesh 2: {len(vertices2_translated)} vertices, {len(all_faces_2)} total faces")
    logging.info(f"Mesh 1 Boundary Faces: {len(boundary_faces_1)}, Internal Faces: {len(internal_faces_1)}")
    logging.info(f"Mesh 2 Boundary Faces: {len(boundary_faces_2)}, Internal Faces: {len(internal_faces_2)}")
    logging.info(f"Touching Faces: {len(touching_faces)}")

    # Visualize the meshes
    visualize_all_with_details(
        vertices1,
        vertices2_translated,
        external_faces_1,
        external_faces_2,
        internal_faces_1,
        internal_faces_2,
        touching_faces
    )


def visualize_all_with_details(vertices1, vertices2, ext_faces_1, ext_faces_2, int_faces_1, int_faces_2, touching_faces):
    """
    Visualize external, internal, and touching faces for two meshes.
    """
    ps.init()
    ps.set_navigation_style("turntable")

    # Mesh 1 external faces
    if ext_faces_1:
        ps.register_surface_mesh(
            "Mesh 1 External",
            vertices1,
            np.array(ext_faces_1),
            color=[0.7, 0.7, 0.7],
            transparency=0.5,
            smooth_shade=True
        )

    # Mesh 1 internal faces
    if int_faces_1:
        ps.register_surface_mesh(
            "Mesh 1 Internal",
            vertices1,
            np.array(int_faces_1),
            color=[0.2, 1.0, 0.2],
            transparency=0.5,
            smooth_shade=True
        )

    # Mesh 2 external faces
    if ext_faces_2:
        ps.register_surface_mesh(
            "Mesh 2 External",
            vertices2,
            np.array(ext_faces_2),
            color=[0.5, 0.5, 1.0],
            transparency=0.5,
            smooth_shade=True
        )

    # Mesh 2 internal faces
    if int_faces_2:
        ps.register_surface_mesh(
            "Mesh 2 Internal",
            vertices2,
            np.array(int_faces_2),
            color=[1.0, 0.2, 0.2],
            transparency=0.5,
            smooth_shade=True
        )

    # Touching faces
    if touching_faces:
        ps.register_surface_mesh(
            "Touching Faces",
            vertices1,
            np.array(touching_faces),
            color=[1.0, 0.0, 0.0],
            smooth_shade=True
        )

    ps.show()


In [40]:
def get_bottom_edges_and_top_points(mesh, z_bottom_threshold, z_top_threshold):
    bottom_edges = []
    top_points = []

    for cell in mesh.cells:
        if cell.type == "line":  # Edges
            for edge in cell.data:
                points = mesh.points[edge]
                if all(point[2] <= z_bottom_threshold for point in points):
                    bottom_edges.append(edge)
        elif cell.type == "vertex":  # Points
            for point_index, point in enumerate(mesh.points):
                if point[2] >= z_top_threshold:
                    top_points.append(point_index)

    return bottom_edges, top_points

In [41]:
z_bottom_threshold = 0.2
z_top_threshold = 1.0

bottom_edges_001, top_points_001 = get_bottom_edges_and_top_points(mesh_cube_001, z_bottom_threshold, z_top_threshold)
bottom_edges_002, top_points_002 = get_bottom_edges_and_top_points(mesh_cube_002, z_bottom_threshold, z_top_threshold)

NameError: name 'mesh_cube_001' is not defined

In [21]:
def main():
    # Update these paths to your mesh files
    file_path1 = os.path.join(os.getcwd(), "../../meshes/rectangle.mesh")
    file_path2 = os.path.join(os.getcwd(), "../../meshes/rectangle.mesh")
    
    if not os.path.exists(file_path1):
        raise FileNotFoundError(f"Mesh 1 file not found: {file_path1}")
    if not os.path.exists(file_path2):
        raise FileNotFoundError(f"Mesh 2 file not found: {file_path2}")
    
    # Load meshes
    vertices1, cells1 = load_mesh(file_path1)
    vertices2, cells2 = load_mesh(file_path2)

    # Define the translation vector
    translation_vector = np.array([12.0, 0.0, 0.0])

    # Translate the second mesh vertices
    vertices2_translated = vertices2 + translation_vector

    # Extract faces for both meshes
    all_faces_1 = extract_faces(cells1)
    all_faces_2 = extract_faces(cells2)

    # Get boundary faces for both meshes
    boundary_faces_1 = extract_boundary_faces(all_faces_1)
    boundary_faces_2 = extract_boundary_faces(all_faces_2)

    # Identify touching faces (intersection of boundary faces)
    set_bf1 = set(boundary_faces_1)
    set_bf2 = set(boundary_faces_2)
    touching_faces = list(set_bf1.intersection(set_bf2))

    # Calculate non-touching exterior faces
    mesh1_ext_faces = list(set_bf1.difference(touching_faces))
    mesh2_ext_faces = list(set_bf2.difference(touching_faces))

    # Internal faces (appear more than once across all tetrahedral cells)
    all_faces_combined = all_faces_1 + all_faces_2
    internal_faces = [
        face for face, count in Counter(all_faces_combined).items() if count > 1
    ]

    # Debug information
    logging.info(f"Mesh 1: {len(vertices1)} vertices, {len(all_faces_1)} total faces, {len(boundary_faces_1)} boundary faces")
    logging.info(f"Mesh 2 (translated): {len(vertices2_translated)} vertices, {len(all_faces_2)} total faces, {len(boundary_faces_2)} boundary faces")
    logging.info(f"Number of touching faces: {len(touching_faces)}")

    # Visualize the meshes
    visualize_all_with_details(
        vertices1,
        vertices2_translated,
        external_faces_1,
        external_faces_2,
        internal_faces_1,
        internal_faces_2,
        touching_faces
    )


In [20]:
if __name__ == "__main__":
    main()